# Caso 6: optimización de un portafolio de servicios

---
## Maestría en Inteligencia Analítica para la Toma de Decisiones


* **Profesor**:
    * *Andrés Medaglia*
* **Asistentes**:
    * *Ariel Rojas*
    * *Carlos Arroyo*

## Instrucciones generales

El primer paso antes de resolver este laboratorio es leer y entender el **enunciado del caso**. Una vez tengas claro el caso, te explicamos la estructura de este laboratorio (los demás laboratorios siguen una estrucutra similar).

Este laboratorio tiene las siguientes secciones:
* **Formulación**: un breve resumen del modelo con notación matemática y descripción de sus componentes
* **Importación de librerías**
* **Creación de parámetros**
* **Modelado**
* **Reporte de Resultados**

Este tipo de actividades se evaluará sobre un total de 100 puntos. Las celdas calificables se distinguen por tener la instrucción `# your code here`. Antes de estas celdas  encontrarás instrucciones y consejos para resolver las preguntas, también el puntaje que le corresponde.

Ten en cuenta que este laboratorio está divido en dos partes. En una primera parte (preguntas 1 a 5) se formulará un modelo matemático como hemos venido haciendo en el curso. En una segunda parte (preguntas 6 a 9) se resolverá el mismo problema usando una librería especializada (Google OR-Tools).  

¡Éxitos!

## Formulación
---

Te presentamos la formulación del caso de la semana de forma resumida. Te recomendamos revisar la formulación una vez hayas leído el enunciado del caso. Es bueno que te familiarices con los elementos de la formulación antes de iniciar la implementación.

### Parámetros
>#### **Conjuntos**
>* $N$: conjunto de nodos (productos actuales, productos nuevos, nodo ficticio)
>* $A$: conjunto de arcos (**ofertas de migración**: producto actual a nuevo, **conteo**: producto nuevo a nodo ficticio)
>
>#### **Parámetros**
>* $b_{i} = \begin{cases} a_i, & \text{si } i \in N \text{ es producto actual: clientes activos en este producto}
    \\ 0, & \text{si } i \in N \text{ es producto nuevo}
    \\ -\sum_{i}a_i, & \text{si } i \in N \text{ es nodo ficticio: total de ofertas a los clientes en todos los productos} \end{cases}$
>
>* $c_{ij} = \begin{cases} p_{ij}r_{ij}, & \text{si } (i,j) \in A \text{ es arco de producto actual a nuevo: (probabilidad de migración)*(incremento del cargo mensual)}
    \\ 0, & \text{si } (i,j) \in A \text{ es arco de producto nuevo a nodo ficticio} \end{cases}$
>
>* $u_{ij} = \begin{cases} \infty, & \text{si } (i,j) \in A \text{ es arco de producto actual a producto nuevo: no hay límite supuerior}
    \\ 25000, & \text{si } (i,j) \in A \text{ es arco de producto nuevo a nodo ficticio} \end{cases}$

### Variables de Decisión
>* $x_{ij} = \begin{cases} \text{cantidad de clientes que se les ofrece migrar de } i \in N \text{ a } j \in N, & \text{si } (i,j) \in A \text{ es arco de producto actual a nuevo}
    \\ \text{total de clientes que se les ofrece migrar hacia el nuevo } i \in N, & \text{si } (i,j) \in A \text{ es arco de producto nuevo al nodo ficticio} \end{cases}$
    
### Restricciones
> 1. **Restricciones de balance**: las unidades que salgan de un nodo menos las unidades que entren al mismo deben ser iguales al parámetro $b_i$ de dicho nodo. En términos del problema, a todos los clientes, según sus productos actuales, se les deben hacer una oferta hacia los productos nuevos. Luego, las ofertas para los productos nuevos deben fluir hacia el nodo ficticio que se asegura que haya una oferta para todos los clientes.
>> `# Para desarrollo del estudiante`
:
> 2. **Restricciones de cota**: las ofertas de migración a los nuevos productos no pueden superar el tope permitido.
>> `# Para desarrollo del estudiante`
>
> 3. **Naturaleza de las variables**
>> `# Para desarrollo del estudiante`

> **Advertencia**: para esta primera formulación (Preguntas 1 a 5) se sigue la convención vista en el vídeo teórico para la oferta/demanda de cada nodo:

> $b_{i} = \begin{cases} > 0, & \text{si } i \in N \text{ es un nodo de oferta}
    \\ 0, & \text{si } i \in N \text{ es un nodo de transbordo}
    \\ < 0, & \text{si } i \in N \text{ es un nodo de demanda} \end{cases}$

> Ten en cuenta que esta convención cambia en la segunda parte del laboratorio (Preguntas 6 a 9) debido a que usamos una librería especializada (Google OR-Tools). Más adelante, encontrarás una advertencia sobre este cambio (Pregunta 8).

### Función Objetivo
> Maximizar el incremento mensual total estimado luego de ofrecer las migraciones de productos a los clientes.
>> `# Para desarrollo del estudiante`


## Importación de librerías
---
En esta práctica usaremos:
* El paquete `pulp` permite crear modelos de optimización, crear variables, añadir restricciones y muchos más. Le asignamos el alias de `lp`.
* El paquete `pandas` es muy útil para el análisis de datos en general. Le asignamos el alias de `pd`.


In [ ]:
import pandas as pd
import pulp as lp

## Creación de parámetros
---

Los datos que necesitamos para esta práctica se encuentran disponibles en el archivo `Soporte Caso 7.xlsx`. Ya se han procesado los datos del enunciado para tener la información en términos de **arcos** y **nodos**.

Es **muy importante** resaltar cómo se manejaron las condiciones sobre las migraciones:
* No se puede realizar ofertas de migración a productos con menos minutos
* No se puede realizar ofertas de migración a productos con menor navegación
* No se puede realizar ofertas de migración a productos con menor cargo mensual

Puesto que el problema se formuló como una red, la forma más natural de cumplir estas condiciones es **eliminando los arcos** de aquellas migraciones que incumplan alguna de las condiciones. Esto es una práctica común cuando se hace formulación en redes y se hace durante la preparación de los datos, antes de resolver el modelo de optimización.

### Lectura del archivo de soporte

Importamos las hojas `Arcos` y `Nodos` del archivo `Soporte Caso 7.xlsx`, donde encontraremos los atributos que definimos en la formulación.

Estas hojas son importadas como objetos `DataFrame` de `pandas`

In [ ]:
archivo = "Soporte Caso 6.xlsx"
arcos = pd.read_excel(io=archivo, sheet_name="Arcos")
nodos = pd.read_excel(io=archivo, sheet_name="Nodos")

### Procesamiento de archivos de soporte

En este paso, se crean los **Conjuntos** y **Parámetros** que se encuentran en los archivos de soporte.

Cabe aclarar que los parámetros $c_{ij}$ ya contienen la multiplicación de las probabilidades $p_{ij}$ por la diferencia de cargos mensuales $r_{ij}$

In [ ]:
# Nodos del grafo
Nodos = list(nodos.i)

# Arcos del grafo
Arcos = [(str(row.i), row.j) for _, row in arcos.iterrows()]

# Demanda/oferta de cada nodo
b = {str(row.i): row.b for _, row in nodos.iterrows()}

# Costo de cada arco
c = {(str(row.i), row.j): round(row.c, 2) for _, row in arcos.iterrows()}

# Flujo máximo de cada arco
u = {(str(row.i), row.j): row.u for _, row in arcos.iterrows()}

**Celda de prueba (0 puntos)**

Es una buena práctica imprimir algunos objetos que contienen los parámetros en la consola luego de crearlos. De esta forma puedes corregir errores y familiarizarte con las estrucutras de datos que se van a utilizar. Puedes hacer estas pruebas en la celda a continuación.

* **Esta celda no es calificable**

In [ ]:
# Aquí puedes explorar los parámetros


## Modelado
---

### Declaración del modelo

**Pregunta 1 (5 puntos)**
* Crea un objeto modelo en PuLP (`lp.LpProblem`) llamado `problema`
* Indica el sentido de la optimización: maximizar o minimizar

In [ ]:
# your code here


In [ ]:
# Esta celda esta reservada para uso del equipo docente

In [ ]:
# Esta celda esta reservada para uso del equipo docente

### Variables de decisión
`# Para desarrollo del estudiante`

**Pregunta 2 (10 puntos)**
* Crea las variables del modelo: `x`, usando el método `lp.LpVariable.dicts()`
* Especifica el nombre de la variable como `'flujo'` con el argumento `name`
* Especifica los índices de las variables con el argumento `indexs`
* Especifica el tipo de variable con el argumento `cat`
* Especifica la cota inferior de las variables en 0

In [ ]:
# your code here


In [ ]:
# Esta celda esta reservada para uso del equipo docente

In [ ]:
# Esta celda esta reservada para uso del equipo docente

### Función Objetivo

> Maximizar el incremento mensual total esperado luego de ofrecer las migraciones de productos a los clientes.
>> `# Para desarrollo del estudiante`

**Pregunta 3 (10 puntos)**
* Crea la función objetivo y agrégala al modelo `problema`

In [ ]:
# your code here


In [ ]:
# Esta celda esta reservada para uso del equipo docente

In [ ]:
# Esta celda esta reservada para uso del equipo docente

### Restricciones

____
**Ejemplo**
> La siguiente restricción: $\sum_{i \in I} a_{ij} x_{ij} \geq 1, \; \forall j \in J$ es equivalente a:
>    * `for j in J:`
>        * `model += lp.lpSum(a[i,j]*x[i,j] for i in I) >= 1, 'R1_'+str(j)`
    
**Advertencia**: En `pulp` no es recomendable sobreescribir restricciones, entonces, si ya creaste una restricción y quieres crearla de nuevo para corregir algo, asegúrate de volver a crear el modelo `problema` desde el principio. (Nosotros haremos esto antes de calificar, no te preocupes)

**Pregunta 4 (20 puntos)**

* Crea la siguiente restricción, asígnale el nombre `'R1_'+str(<indice_del_para_todo>)` y añádela al modelo:

> 1. **Restricciones de balance**: las unidades que salgan de un nodo menos las unidades que entren al mismo deben ser iguales al parámetro $b_i$ de dicho nodo. En términos del problema, a todos los clientes, según sus productos actuales, se les deben hacer una oferta hacia los productos nuevos. Luego, las ofertas para los productos nuevos deben fluir hacia el nodo ficticio que se asegura que haya una oferta para todos los clientes.
>> `# Para desarrollo del estudiante`

* **Nota**: La creación de estas restricciones puede tomar algunos segundos

* **Advertencia**: para esta primera formulación (Preguntas 1 a 5) se sigue la convención vista en el vídeo teórico para la oferta/demanda de cada nodo:

> $b_{i} = \begin{cases} > 0, & \text{si } i \in N \text{ es un nodo de oferta}
    \\ 0, & \text{si } i \in N \text{ es un nodo de transbordo}
    \\ < 0, & \text{si } i \in N \text{ es un nodo de demanda} \end{cases}$

> Ten en cuenta que esta convención cambia en la segunda parte del laboratorio (Preguntas 6 a 9) debido a que usamos una librería especializada (Google OR-Tools). Más adelante, encontrarás una advertencia sobre este cambio (Pregunta 8).

In [ ]:
# your code here


In [ ]:
# Esta celda esta reservada para uso del equipo docente

**Pregunta 5 (5 puntos)**

* Crea la siguiente restricción, asígnale el nombre `'R2_'+str(<indice_del_para_todo>)` y añádela al modelo:

> 2. **Restricciones de cota**: las ofertas de migración a los nuevos productos no pueden superar el tope permitido.
>> `# Para desarrollo del estudiante`

**Nota:** en caso de usar arcos, no debes concatenar el origen y el destino. Trata el par $(i, j)$ como un único índice.

In [ ]:
# your code here


In [ ]:
# Esta celda esta reservada para uso del equipo docente

In [ ]:
# Esta celda esta reservada para uso del equipo docente

### Invocar el optimizador

El modelo puede tomar algunos segundos en correr.
Cabe resaltar que, a pesar de haber definido las variables como continuas, dada la estructura de red de la formulación y los parámetros enteros del problema ($b_i,u_{ij}$), podemos esperar soluciones enteras sin necesidad de recurrir a *Branch & Bound*! El modelo correrá en muy poco tiempo.

In [ ]:
print(lp.LpStatus[problema.solve()])

## Reporte de resultados
---

**Función objetivo**

In [ ]:
obj = lp.value(problema.objective)
print(f"\nIngreso Adicional Mensual Esperado = {obj / 1e6: .3f} Millones de USD")

**Número total de ofertas para cada producto del nuevo portafolio**

In [ ]:
total = {}
for i in Nodos:
    if i[-1] == "_":
        val = x[i, "Total"].value()
        total[i, "Total"] = val
        print(f"Producto {i} -> {val}")

**Ofertas a los clientes de los productos actuales**

En el siguiente `DataFrame` se pueden consultar las ofertas que se hacen a los clientes de cada producto.
En las **filas** se tienen los productos actuales y en las **columnas** se tienen los productos del nuevo portafolio.
Cada celda puede ser:
* `<int>`: El número indica la cantidad de ofertas del producto actual (fila) para migrar al producto del nuevo portafolio (columna)
* `-`: Significa que no se hacen ofertas para esta migración (toma valor de $0$)
* `X`: Significa que esta migración se eliminó por la condición sobre los minutos, navegación o cargo mensual.

In [ ]:
matrix, matrix2 = [], []
for i in [a for a in Nodos if ((a[-1] != "_") & (a != "Total"))]:
    row, row2 = [], []
    for j in [b for b in Nodos if b[-1] == "_"]:
        e = (i, j)
        if e in Arcos:
            if x[e].value() > 0:
                row.append(x[e].value())
                row2.append(x[e].value())
            else:
                row.append("-")
                row2.append(0)
        else:
            row.append("X")
            row2.append(0)
    matrix.append(row)
    matrix2.append(row2)

df = pd.DataFrame(
    matrix,
    index=[a for a in Nodos if ((a[-1] != "_") & (a != "Total"))],
    columns=[b for b in Nodos if b[-1] == "_"],
)
df.head()

**Productos actuales con ofertas a múltiples productos del nuevo portafolio**

In [ ]:
a1, a2 = [], []
for i, j in Arcos:
    if j[-1] == "_":
        if x[i, j].value() > 0 and x[i, j].value() != b[i]:
            if i not in a1:
                a1.append(i)
            if j not in a2:
                a2.append(j)
df.loc[a1, a2]

## Uso de Google OR-Tools
---
Queremos usar este problema para ilustrar el uso de herramientas diferentes a `PuLP` para resolver problemas con herramientas especializadas en menor tiempo.

En este caso, tenemos un problema de **Flujo de costo mínimo**. Ya vimos que lo podemos resolver con las herramientas que hemos venido utilizando. Sin embargo, este problema se ha estudiado por varias disciplinas (e.g., teoría de grafos) y existen implementaciones en Python con mejoras algorítmicas sustanciales.

**Google OR-Tools** es una API de acceso público de Google para resolver problemas de optimización. Cuentan con un algoritmo especializado en problemas de **Flujo de costo mínimo el cual** utilizaremos a continuación. Cuando usamos una nueva herramienta, es útil leer algo de documentación: https://developers.google.com/optimization/flow/mincostflow. Así conoceremos la forma correcta de entregarle los datos a nuestro algoritmo y qué comandos usar para construir el modelo.

### Preprocesamiento

El primer paso es procesar los datos al formato que espera nuestra herramienta.
Luego de leer la documentación, comprobamos que debemos crear los nodos y arcos uno a uno con sus respectivos atributos y que los nodos solo pueden tener `int` como nombres, no se permite `str`. Por esto debemos crear dos diccionarios que nos permitan asignar nombres `int` a los nombres de los nodos que tenemos actualmente y viceversa:

In [ ]:
nodos_dict = {i: name for name, i in enumerate(b.keys())}
nodos_dict2 = {name: i for name, i in enumerate(b.keys())}

Para facilitar la creación de nodos y arcos en esta herramienta, se deben crear listas de tuplas con los nombres y atributos de nodos y arcos.


**Pregunta 6 (5 puntos)**

* Crea una lista de nodos `lista_nodos` donde cada elemento debe ser una tupla `(nodos_dict[i], -b_i)`, que contiene el nombre codificado y el parámetro de oferta/demanda correspondiente de cada nodo

> **Advertencia:** Nota que la tupla contiene a `-b_i` con signo negativo. Esto es porque las restricciones los signos de las restricciones de balance están invertidos en esta herramienta respecto a la formulación que hicimos anteriormente.

In [ ]:
# your code here


In [ ]:
# Esta celda esta reservada para uso del equipo docente

In [ ]:
# Esta celda esta reservada para uso del equipo docente

**Pregunta 7 (10 puntos)**

* Crea una lista de arcos `lista_arcos` donde cada elemento debe ser una tupla `(nodos_dict[i],nodos_dict[j],u_ij,-int(c_ij*100))`, que contiene el nombre codificado de los nodos del arco, la capacidad del arco y el costo unitario del arco respectivamente.

> **Advertencia:** Nota que la tupla contiene a `-int(c_ij*100)`, en lugar de `c_ij`. Esto es por dos razones. Primero, el sentido del problema anterior era de maximización; el algoritmo que usaremos es por defecto de minimización. Notese que es equivalente maximizar un costo a minimizar el mismo costo con signo negativo. Esto explica el signo negativo. Segundo, OR-Tools no permite valores continuos `float` en el costo de los arcos. Por esto, debemos multiplicar por 100 para conservar los 2 decimales antes de convertir a `int`. Luego de resolver el problema, dividiremos nuevamente por esta cantidad para obtener el costo correcto sin perder decimales en el camino.

In [ ]:
# your code here


In [ ]:
# Esta celda esta reservada para uso del equipo docente

In [ ]:
# Esta celda esta reservada para uso del equipo docente

### Modelado

Para empezar, importamos la librería correspondiente y creamos el objeto modelo vacío.

In [ ]:
from ortools.graph.python import min_cost_flow

mcf = min_cost_flow.SimpleMinCostFlow()

**Pregunta 8 (15 puntos)**

El siguiente paso es **crear los nodos**.

Para esto se usará el método `mcf.set_node_supply(node, supply)`

* Dentro de un bucle `for` recorre los elementos de `lista_nodos`. Por cada elemento usa el método `mcf.set_node_supply` y especifica los argumentos `node` y `supply` como el nombre codificado y el parámetro oferta/demanda respectivamente.

In [ ]:
# your code here


In [ ]:
# Esta celda esta reservada para uso del equipo docente

**Pregunta 9 (20 puntos)**

Ahora debemos **crear los arcos**.

Para esto se usará el método `mcf.add_arc_with_capacity_and_unit_cost(head, tail, capacity, unit_cost)`

* Dentro de un bucle `for` recorre los elementos de `lista_arcos`. Por cada elemento usa el método `mcf.add_arc_with_capacity_and_unit_cost` y especifica los argumentos `head`, `tail`, `capacity` y `unit_cost` como los nombres codificados de los nodos, la capacidad del arco y el costo unitario del arco respectivamente.

In [ ]:
# your code here


In [ ]:
# Esta celda esta reservada para uso del equipo docente

In [ ]:
# Esta celda esta reservada para uso del equipo docente

¡Ahora solo queda invocar el algoritmo!

In [ ]:
status = mcf.solve()
print(status == mcf.OPTIMAL)

### Reporte de resultados

Podemos verificar a continuación que el valor de la función objetivo es igual a la implementación anterior con `PuLP`.

**Función objetivo**

In [ ]:
obj = -mcf.optimal_cost() / 1e2
print(f"\nIngreso Adicional Mensual Esperado = {obj / 1e6: .3f} Millones de USD")

Antes de revisar los valores de las variables, decodificamos los flujos a los nombres que hemos venido usando anteriormente

In [ ]:
flow = {
    (
        nodos_dict2[mcf.head(arc)],
        nodos_dict2[mcf.tail(arc)],
    ): mcf.flow(arc)
    for arc in range(mcf.num_arcs())
}

**Número total de ofertas para cada producto del nuevo portafolio**

In [ ]:
total = {}
for i in Nodos:
    if i[-1] == "_":
        val = flow[i, "Total"]
        total[i, "Total"] = val
        print(f"Producto {i} -> {val}")

**Ofertas a los clientes de los productos actuales**

In [ ]:
matrix, matrix2 = [], []
for i in [a for a in Nodos if ((a[-1] != "_") & (a != "Total"))]:
    row, row2 = [], []
    for j in [b for b in Nodos if b[-1] == "_"]:
        e = (i, j)
        if e in Arcos:
            if flow[e] > 0:
                row.append(flow[e])
                row2.append(flow[e])
            else:
                row.append("-")
                row2.append(0)
        else:
            row.append("X")
            row2.append(0)
    matrix.append(row)
    matrix2.append(row2)

df = pd.DataFrame(
    matrix,
    index=[a for a in Nodos if ((a[-1] != "_") & (a != "Total"))],
    columns=[b for b in Nodos if b[-1] == "_"],
)
df.head()

**Productos actuales con ofertas a múltiples productos del nuevo portafolio**

In [ ]:
a1, a2 = [], []
for i, j in Arcos:
    if j[-1] == "_":
        if flow[i, j] > 0 and flow[i, j] != b[i]:
            if i not in a1:
                a1.append(i)
            if j not in a2:
                a2.append(j)
df.loc[a1, a2]

### Fin del laboratorio
---